In [1]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import io
import time
import numpy as np
import cv2
import os
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import NoSuchWindowException

In [5]:
region=['강화군','계양구','남동구','동구','미추홀구','부평구','서구','연수구','용진군','중구']
keward_text=[]
store=[]

In [8]:
keward_text=[]
store=[]
keward_text2=[]
store2=[]

In [4]:
ganghwa_gun=['강화읍','교동면','길상면','내가면','불은면','삼산면',
            '선원면','송해면','양도면','양사면','하점면','화도면']# 서도면 없음
geyang_gu=['계산동','계양동','작전동','작전서운동','효성동']
namdong_gu=['간석동','구월동','도림동'
           '남촌도림동','논현동','논현고잔동',
           '만수동','서창2동','장수서창동']
dong_gu=['금창동','만석동','송림동',
        '송현동','화수1.화평동','화수동']
michuhol_gu = [
    "관교동", "도화동", "문학동", "숭의동",
    "용현동", "주안동", "학익동"]
bupeyong_gu =[
    "갈산동", "부개동","부평동", "산곡동", "삼산동", "십정동", "일신동", "청천동"]
west_gu = [
    "가정동", "가좌동",
    "검단동", "검암경서동", "당하동", "마전동", "불로대곡동", "석남동",
    "신현원창동", "아라동", "연희동", "오류왕길동","원당동", "청라동"]
yeonsu_gu = ["동춘동", "선학동", "송도동", "연수동", "옥련동", "청학동"]
yonggin_gun = ["백령면", "북도면", "영흥면"]#백령면,북도면,영흥면 말고 카페x
jong_gu = ["개항동", "도원동", "동인천동", "신포동", "신흥동", "연안동", "영종동","용유동", "운서동", "율목동"]


In [5]:
region_detail=[ganghwa_gun,geyang_gu,namdong_gu,dong_gu,michuhol_gu,bupeyong_gu,west_gu,yeonsu_gu,yonggin_gun,jong_gu]
flat_list = [item for sublist in region_detail for item in sublist]

In [6]:
def page_down(num):
    body = browser.find_element(By.CSS_SELECTOR, 'body')
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)
def page_down_detail():
    element = browser.find_elements(By.CSS_SELECTOR, 'body')
    element=element[1]
    element.send_keys(Keys.END)  
def search_iframe():
    browser.switch_to.default_content()
    browser.switch_to.frame("searchIframe")

def entry_iframe():
    browser.switch_to.default_content()
    WebDriverWait(browser, 5).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

In [7]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome()
for dong in flat_list:
    browser.get('https://map.naver.com/p?c=15.00,0,0,0,dh')
    time.sleep(2)
    incheon_input=browser.find_elements(By.CSS_SELECTOR,'input[type=text]')
    time.sleep(1)
    incheon_input[1].send_keys(f'인천광역시 {dong} 카페')
    incheon_input[1].send_keys(Keys.ENTER)
    # 카페 이름 수집
    for i in range(1, 13):
                time.sleep(2)
                search_iframe()
                time.sleep(1)
                try:
                    name_link = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                                f'#_pcmap_list_scroll_container > ul > li:nth-child({i}) > div.CHC5F > a.tzwk0 > div > div > span.place_bluelink.TYaxT')))
                    browser.execute_script("arguments[0].click();", name_link)
                except:
                    break
                time.sleep(1)
                try:
                    entry_iframe()
                except:
                    pass
                # "리뷰 더보기" 버튼을 찾고 클릭
                try:
                    element2 = browser.find_element(By.XPATH, '//span[text()="리뷰"]')
                    browser.execute_script("arguments[0].click();", element2)
                except:
                    pass
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight/4);")
                for k in range(6):
                    page_source = browser.page_source
                    bs = BeautifulSoup(page_source, 'html.parser')
                    start_time = time.time()  # 현재 시간 기록
                    while True:
                        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
                        try:
                            # 요소가 보일 때까지 기다림 (타임아웃은 10초로 설정)
                            element3 = WebDriverWait(browser, 10).until(
                                EC.visibility_of_element_located((By.XPATH, '//a[@class="fvwqf"]')))
                            # 요소가 보이면 반복문 종료
                            break
                        except:
                            # 요소가 아직 보이지 않으면 계속 스크롤
                            break
                        if time.time() - start_time > 5:
                            break
                    try:
                        browser.execute_script("arguments[0].click();", element3)
                    except:
                        break
                allDiv=bs.find_all('li',class_='owAeM')
                for review_box in allDiv:
                    store.append(f'{dong}')   
                    try:
                        keward_box = review_box.find_all('div',class_='ERkm0')
                        for k in keward_box:
                            try:
                                keward = k.find('span')
                                keward_text.append(keward.text)
                            except:
                                keward_text.append('평가 없음')
                    except:
                        keward_text.append('평가 없음')

In [12]:
cafe_df_keward = pd.DataFrame(data=[], columns=['id','keward'])
# k개의 리뷰를 수집합니다.
for i in range(len(keward_text)):
    tmp = []
    tmp.append(store[i])
    tmp.append(keward_text[i])
    tmp = pd.DataFrame(data=[tmp], columns = cafe_df_keward.columns)
    cafe_df_keward = pd.concat([cafe_df_keward,tmp])
    

cafe_df_keward.reset_index(inplace=True, drop=True)
cafe_df_keward

,id,keward
0,강화읍,인테리어가 멋져요
1,강화읍,커피가 맛있어요
2,강화읍,커피가 맛있어요
3,강화읍,인테리어가 멋져요
4,강화읍,커피가 맛있어요
...,...,...
11827,운서동,커피가 맛있어요
11828,운서동,디저트가 맛있어요
11829,운서동,커피가 맛있어요
11830,운서동,커피가 맛있어요


In [14]:
cafe_df_keward.to_csv('cafe_df_keward.csv',index=False, encoding='utf-8-sig')

In [10]:
for region_id in region:
    for dong in flat_list:
        print(f'{dong}')

강화읍
교동면
길상면
내가면
불은면
삼산면
선원면
송해면
양도면
양사면
하점면
화도면
계산동
계양동
작전동
작전서운동
효성동
간석동
구월동
도림동남촌도림동
논현동
논현고잔동
만수동
서창2동
장수서창동
금창동
만석동
송림동
송현동
화수1.화평동
화수동
관교동
도화동
문학동
숭의동
용현동
주안동
학익동
갈산동
부개동
부평동
산곡동
삼산동
십정동
일신동
청천동
가정동
가좌동
검단동
검암경서동
당하동
마전동
불로대곡동
석남동
신현원창동
아라동
연희동
오류왕길동
원당동
청라동
동춘동
선학동
송도동
연수동
옥련동
청학동
백령면
북도면
영흥면
개항동
도원동
동인천동
신포동
신흥동
연안동
영종동
용유동
운서동
율목동
강화읍
교동면
길상면
내가면
불은면
삼산면
선원면
송해면
양도면
양사면
하점면
화도면
계산동
계양동
작전동
작전서운동
효성동
간석동
구월동
도림동남촌도림동
논현동
논현고잔동
만수동
서창2동
장수서창동
금창동
만석동
송림동
송현동
화수1.화평동
화수동
관교동
도화동
문학동
숭의동
용현동
주안동
학익동
갈산동
부개동
부평동
산곡동
삼산동
십정동
일신동
청천동
가정동
가좌동
검단동
검암경서동
당하동
마전동
불로대곡동
석남동
신현원창동
아라동
연희동
오류왕길동
원당동
청라동
동춘동
선학동
송도동
연수동
옥련동
청학동
백령면
북도면
영흥면
개항동
도원동
동인천동
신포동
신흥동
연안동
영종동
용유동
운서동
율목동
강화읍
교동면
길상면
내가면
불은면
삼산면
선원면
송해면
양도면
양사면
하점면
화도면
계산동
계양동
작전동
작전서운동
효성동
간석동
구월동
도림동남촌도림동
논현동
논현고잔동
만수동
서창2동
장수서창동
금창동
만석동
송림동
송현동
화수1.화평동
화수동
관교동
도화동
문학동
숭의동
용현동
주안동
학익동
갈산동
부개동
부평동
산곡동
삼산동
십정동
일신동
청천동
가정동
가좌동
검단동
검암경서동
당하동
마전동
불로대곡동
석남동
신현원창동
아라동
연희동
오류왕길동
원당동
청라동
동춘동
선학동
송도동
연수동
옥련동
청학동
백령면
북도면
영흥면
개항동
도원동
동인천동
신포동
신

In [11]:
cafe_df_keward['id'].value_counts()

id
주안동    250
갈산동    230
화도면    220
학익동    210
불은면    210
      ... 
도원동     80
만석동     80
길상면     50
십정동     30
운서동     14
Name: count, Length: 73, dtype: int64

In [25]:
add_list=['남촌도림동',  '영종1동', '부개1동']


In [26]:
keward_text=[]
store=[]
review_txt=[]

In [27]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome()
for region_id in add_list:
    browser.get('https://map.naver.com/p?c=15.00,0,0,0,dh')
    time.sleep(2)
    incheon_input=browser.find_elements(By.CSS_SELECTOR,'input[type=text]')
    time.sleep(1)
    incheon_input[1].send_keys(f'인천광역시 {region_id} 카페')
    incheon_input[1].send_keys(Keys.ENTER)

    # 카페 이름 수집
    time.sleep(1)
    for i in range(1, 9):
        search_iframe()
        time.sleep(2)
        name_link = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
                                                                        f'#_pcmap_list_scroll_container > ul > li:nth-child({i}) > div.CHC5F > a.tzwk0 > div > div > span.place_bluelink.TYaxT')))
        browser.execute_script("arguments[0].click();", name_link)
        time.sleep(1)
        entry_iframe()

        # "리뷰 더보기" 버튼을 찾고 클릭
        element2 = browser.find_element(By.XPATH, '//span[text()="리뷰"]')
        browser.execute_script("arguments[0].click();", element2)
        time.sleep(1)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/4);")
        for k in range(20):
            page_source = browser.page_source
            bs = BeautifulSoup(page_source, 'html.parser')
            time.sleep(1)
            start_time = time.time()  # 현재 시간 기록
            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
                try:
                    # 요소가 보일 때까지 기다림 (타임아웃은 10초로 설정)
                    element3 = WebDriverWait(browser, 10).until(
                        EC.visibility_of_element_located((By.XPATH, '//a[@class="fvwqf"]')))
                    # 요소가 보이면 반복문 종료
                    break
                except:
                    # 요소가 아직 보이지 않으면 계속 스크롤
                    break
                if time.time() - start_time > 5:
                    break
            try:
                browser.execute_script("arguments[0].click();", element3)
            except:
                break
        allDiv=bs.find_all('li',class_='owAeM')
        for review_box in allDiv:
            store.append(f'{region_id}')      
            try:
                review_text2 = review_box.find_all('div',class_='vg7Fp CyA_N')
                for j in review_text2:
                    review_bs=j.find('span',class_='zPfVt')
                    review_txt.append(review_bs.text) 
            except:
                review_txt.append('평가 없음')
            try:
                keward_box = review_box.find_all('div',class_='ERkm0')
                for k in keward_box:
                    try:
                        keward = k.find('span')
                        keward_text.append(keward.text)
                    except:
                        keward_text.append('평가 없음')
            except:
                keward_text.append('평가 없음')

cafe_df_reivew_keward = pd.DataFrame(data=[], columns=['id','review','keward'])

In [28]:
add_keward = pd.DataFrame(data=[], columns=['id','review','keward'])
# k개의 리뷰를 수집합니다.
for i in range(len(review_txt)):
    tmp = []
    tmp.append(store[i])
    tmp.append(review_txt[i])
    tmp.append(keward_text[i])
    tmp = pd.DataFrame(data=[tmp], columns = add_keward.columns)
    add_keward = pd.concat([add_keward,tmp])
    

add_keward.reset_index(inplace=True, drop=True)
add_keward

,id,review,keward
0,남촌도림동,빵종류 다양하고 주차넓고 커피도 맛있어요. 야외자리도 많아서 날씨 좋은날 오면 정말...,빵이 맛있어요
1,남촌도림동,아이와 힐링하러왔어요 즐거운 시간이였어요,뷰가 좋아요
2,남촌도림동,오랜만에 왔는데 빵 종류도 더 다양해지고 좋아졌어요! 분위기도 좋고 빵이랑 커피 너...,빵이 맛있어요
3,남촌도림동,날씨가 좋아서 야외에서 음료랑 디저트 먹기 좋은 장\n소네요 가족모임후 여럿 왔는데...,빵이 맛있어요
4,남촌도림동,우연치않게 지나가다 들른 카페인데 커피도 빵도 맛\n있었고 분위기가 좋았던 곳이네요...,빵이 맛있어요
...,...,...,...
2905,부개1동,"브랜드도 아니고, 체인점도 아닌데, 수제빵집인데도, 맛있게 잘하니 고객들이 많고, ...",친절해요
2906,부개1동,좋아요,특별한 메뉴가 있어요
2907,부개1동,직원분도 친절하시고 빵도 맛있어요,인테리어가 멋져요
2908,부개1동,긋,음료가 맛있어요


In [29]:
add_keward.to_csv('add_keward4.csv',index=False, encoding='utf-8-sig')

In [14]:
review_txt

['아이들이 더 좋아해서 자주와요. 다른빵집은 가고 싶\n은 생각이 없어지는곳 너무 맛있어서 다른사람에게 \n꼭 선물하고 싶은곳이에요 지금처럼 오래오래 ...',
 '다른빵도 맛있지만 산딸기바게트 너무 좋아하고 맛있\n어서 두개 사서 왔어요',
 '친구 집에 놀러가면서 사갔는데 다 맛있다고 좋아했어요!',
 '빵이 맛있다는 소문듣고 찾아간곳 역시 소문대로 맛있게 잘 먹었습니다~~\n사진엔 없지만 바게트 샌드위치도 넘넘 맛있네요~~!!',
 '🥐🥖🥪🍪두번째방문\n밀크롤 없어서 아쉽 ㅠㅠ',
 '저녁 때 가서 빵이 많이 없었는데 그래도 잔뜩 담았네요. ',
 '우유식빵,포함 다섯개 구입  소금빵 두개 포함입니다 \n가격이  비싸긴 하네요 ',
 '빵이전체적으로 좀 심심해요..',
 '산곡동 맛집 검색하면 왜 빵집(쉐라메르)이 나오는지 의아했으나 한번 가본 결과 너무나 납득해버림. 디저트 빵에 있어서는 최강자임 ',
 '과자종류 사고 치즈빵 사고',
 '맛있다..🥰증맬루..✨',
 '몽블랑이 달달하고 부드러워서 어른들도 좋아하시고 맛있어요~ 밀크롤, 산딸기바게트, 명란바게트 등 맛있는 빵이 많은 곳이에요!',
 '우리동네 빵집 맛집 비싸지만 진짜 맛있다',
 '빵이너무맛있어서 밥먹고갔는데도순삭했어요.',
 '여기는 진짜 손에 꼽히는 빵 맛집',
 '단골입니다. 이사와서 자주 못가지만 본가갈 때마다 가요. 직원분들 친절하시고 요기요 배달에도 떠서 좋네요.',
 '케이크 너무 맛있어용 여기만 먹어용',
 '산딸기 바게트 맛있어요!\n이탈리안 고로케? 그것도 맛있어요!!!\n명란바게트는 생각보다 별로였어요',
 '산딸기브레드 넘맛있어여😍',
 '동네 빵 맛집! 프랜차이즈 빵집보다 훨씬 맛있고 종류도 다양해요',
 '선물용으로도 많이 사는데 선물받는 분들 모두 맛있\n다고 하세요! 몽블랑은 이번에 처음 사봤는데 아주 맛\n있었어요~ 달달하고 부드러워서 어르신들도 좋...',
 '우유식빵, 옥수수스콘. 단팥빵, 치즈빵 샀어요',
 '코로나로 시식은 없어졌나봐요. 여전히 빵은